# CART Decision Tree Classifier from Scratch

In [1]:
import pandas as pd
import numpy as np
eps = np.finfo(float).eps
from anytree import Node, NodeMixin, RenderTree
from anytree.importer import DictImporter
from numpy import log2 as log
import matplotlib.pyplot as plt
import seaborn as sns
import math
from collections import Counter
from pprint import pprint
import json
import pydot
%matplotlib inline

In [2]:
# Load data file
bank = pd.read_csv('./data/bank-full.csv', sep=';')
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
bank.shape

(45211, 17)

In [4]:
# Missing values: None 
bank[bank.isnull().any(axis=1)].count()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [5]:
numerical_features = list(bank.describe().columns) # numerical features
bank.describe() # Describes the features that are numerical

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [6]:
categorical_features = []
for feature in bank.columns.tolist():
    if (feature not in numerical_features and feature != 'y'):
        categorical_features.append(feature) # categorical features
bank.drop(numerical_features, axis = 1, inplace = True)
features = list(bank.drop(['y'], axis = 1, inplace = False).columns) # independent variables

## The CART Algorithm

In [7]:
# Takes a list of items with discrete values and returns the gini index for those items.
def gini_index(a_list):
    cnt = Counter(x for x in a_list)
    num_instances = len(a_list)*1.0
    probs = [x / num_instances for x in cnt.values()]
    return 1 - sum([prob ** 2 for prob in probs])
    
gini_job = gini_index(bank['job'])
# print(gini_job)

In [8]:
# Takes a DataFrame of attributes, and quantifies the weighted average of the gini index 
# of a target attribute after performing a split along the values of another attribute.

def weighted_average_gini(df, split_attribute_name, target_attribute_name, trace=0):
    df_split = df.groupby(split_attribute_name)
    nobs = len(df.index) * 1.0
    df_agg_ent = df_split.agg({target_attribute_name : [gini_index, lambda x: len(x)/nobs] })[target_attribute_name]
    df_agg_ent.columns = ['Gini Index', 'PropObservations']
    if trace: 
        print(df_agg_ent)
        
    wtd_avg_gini = sum(df_agg_ent['Gini Index'] * df_agg_ent['PropObservations'])
    return wtd_avg_gini

# print('\nWeighted Average Gini for the poutcome attribute is ' + str(weighted_average_gini(bank, 'poutcome', 'y')))

In [9]:
def CART(df, target_attribute_name, attribute_names, default_class=None):
    cnt = Counter(x for x in df[target_attribute_name])
    
    if len(cnt) == 1: # Leaf / Homogenous node
        return list(cnt.keys())[0]
    
    elif df.empty or (not attribute_names): # Data instance empty?
        return default_class 
    
    else:
        index_of_min = list(cnt.values()).index(min(cnt.values())) 
        default_class = list(cnt.keys())[index_of_min]
        
        gini = [weighted_average_gini(df, attr, target_attribute_name) for attr in attribute_names]
        index_of_min = gini.index(min(gini)) 
        best_attr = attribute_names[index_of_min]
        
        tree = {best_attr:{}}
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = CART(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree

In [10]:
tree = CART(bank, 'y', features)

In [11]:
# print(treeify(tree))

## Classification

In [12]:
def classify(instance, tree, default=None):
    attribute = list(tree.keys())[0]
    if instance[attribute] in tree[attribute].keys():
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict):
            return classify(instance, result)
        else:
            return result # this is a label
    else:
        return default

In [13]:
def accuracy(train_tree, algo):
    test_data[algo] = test_data.apply(classify, axis=1, args=(train_tree, 'yes'))
    print('Accuracy is ' + str(sum(test_data['y']==test_data[algo] ) / (1.0*len(test_data.index))))
    return

In [14]:
def confusion_matrix(test_data, algo): 
    predictions = str(algo + '_predicted')
    df = test_data[['y', predictions]]
    matrix = pd.DataFrame([[0,0],[0,0]])
    label_encoder = {0 : 'no', 1 : 'yes'} # Class 0: No, Class 1: Yes
    for i in label_encoder.keys():
        for j in label_encoder.keys():
            count = df[(df['y'] == label_encoder[i]) & (df[predictions] == label_encoder[j])].shape[0]
            matrix.iloc[i, j] = count
    return matrix

In [15]:
def accuracy_metrics(matrix):
    TN = matrix.iloc[0,0]; FP = matrix.iloc[0,1]; FN = matrix.iloc[1,0]; TP = matrix.iloc[0,0];
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F1 = 2*TP/(2*TP+FP+FN)
    print("precision: {0}, recall: {1}, F1-score:{2}".format(precision, recall, F1))
    return 

In [16]:
training_data = bank.iloc[0:36169, :] #  80% of the entire data goes for training
test_data = bank.iloc[36169:, :]      #  20% of the entire data goes for testing

In [17]:
CART_train_tree = CART(training_data, 'y', features)
accuracy(CART_train_tree, 'CART_predicted')

Accuracy is 0.4107498341074983


/Users/ramanshgrover/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
confusion_matrix(test_data, 'CART')

,0,1
0,2672,2596
1,1072,1042


In [19]:
accuracy_metrics(confusion_matrix(test_data, 'CART'))

precision: 0.507213363705391, recall: 0.7136752136752137, F1-score:0.5929871282734133


# Comparison

In [20]:
import import_ipynb
import C45
import ID3

importing Jupyter notebook from C45.ipynb


C45.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {


Accuracy is 0.5738774607387747
importing Jupyter notebook from ID3.ipynb


ID3.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {


Accuracy is 0.5747622207476222


## ID3

In [21]:
id3_train_tree = ID3.id3(training_data, 'y', features)
accuracy(id3_train_tree, 'ID3_predicted')

Accuracy is 0.5747622207476222


/Users/ramanshgrover/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
confusion_matrix(test_data, 'ID3')

,0,1
0,4635,635
1,1555,562


In [23]:
accuracy_metrics(confusion_matrix(test_data, 'ID3'))

precision: 0.8795066413662239, recall: 0.7487883683360258, F1-score:0.8089005235602095


## C4.5

In [24]:
c45_train_tree = C45.c45(training_data, 'y', features)
accuracy(c45_train_tree, 'C4.5_predicted')

Accuracy is 0.5738774607387747


/Users/ramanshgrover/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
confusion_matrix(test_data, 'C4.5')

,0,1
0,4628,623
1,1566,561


In [26]:
accuracy_metrics(confusion_matrix(test_data, 'C4.5'))

precision: 0.8813559322033898, recall: 0.7471746851792057, F1-score:0.8087374399301005


# Conclusion

| Algorithm | Precision | Recall (Sensitivity) | F1-score | Prediction Accuracy |
| :-------: | :----------------: | :------------: | :------------: | :------------: |
|    C4.5   |     0.8813559322033898    | 0.7471746851792057 | 0.8087374399301005 | 0.5747622207476222 |
|    ID3    |  0.8795066413662239 | 0.7487883683360258 | 0.8089005235602095  | 0.4107498341074983 |
|    CART   |  0.507213363705391 | 0.7136752136752137 | 0.5929871282734133 | 0.5738774607387747 |

The ID3 Algorithm performed better overall.


# Differences

| Algorithm | Splitting Criteria | Attribute Type | Missing Values | Pruning Strategy | Outlier Detection |
| --------- | ------------------ | -------------- | -------------- | ---------------- | ----------------- |
|    C4.5   |     Gain Ratio     | Categorical & Numerical Values | Does handle | Error Based | Susceptible |
|    ID3    |  Information Gain  | Categorical Values only | Does NOT handle  | No pruning is done | Susceptible |
|    CART   |  Towering Criteria | Categorical & Numerical Values | Does handle | Cost-Complexity | Can Handle |


# Trees (Plotted)

In [27]:
# working on a smaller sample for a comprehensible decision tree
sample = bank.sample(50)

In [28]:
def treeify(tree):
    tree_str = json.dumps(tree, indent=4)
    tree_str = tree_str.replace("\n    ", "\n")
    tree_str = tree_str.replace('"', "")
    tree_str = tree_str.replace(',', "")
    tree_str = tree_str.replace("{", "")
    tree_str = tree_str.replace("}", "")
    tree_str = tree_str.replace("    ", " | ")
    tree_str = tree_str.replace("  ", " ")
    return tree_str

In [29]:
def draw(parent_name, child_name):
    edge = pydot.Edge(parent_name, child_name)
    graph.add_edge(edge)

def visit(node, parent=None):
    for k,v in node.items():
        if isinstance(v, dict):
            # We start with the root node whose parent is None
            # we don't want to graph the None node
            if parent:
                draw(parent, k)
            visit(v, k)
        else:
            draw(parent, k)
            # drawing the label using a distinct name
            draw(k, k+'_'+v)

## The ID3 Tree

In [30]:
id3_tree = ID3.id3(sample, 'y', features)
print(treeify(id3_tree))


job: 
 | admin.: 
 | | marital: 
 | | | married: 
 | | | | month: 
 | | | | | jun: 
 | | | | | | housing: 
 | | | | | | | no: yes
 | | | | | | | yes: no
 | | | | | | 
 | | | | | 
 | | | | | may: no
 | | | | | nov: 
 | | | | | | education: 
 | | | | | | | primary: no
 | | | | | | | secondary: yes
 | | | | | | 
 | | | | | 
 | | | | 
 | | | 
 | | | single: no
 | | 
 | 
 | blue-collar: 
 | | month: 
 | | | apr: no
 | | | jan: no
 | | | jul: no
 | | | jun: no
 | | | may: no
 | | | nov: no
 | | | sep: yes
 | | 
 | 
 | housemaid: no
 | management: no
 | retired: yes
 | self-employed: no
 | services: 
 | | marital: 
 | | | divorced: yes
 | | | married: no
 | | | single: no
 | | 
 | 
 | student: 
 | | housing: 
 | | | no: yes
 | | | yes: no
 | | 
 | 
 | technician: 
 | | contact: 
 | | | cellular: yes
 | | | unknown: no
 | | 
 | 
 | unemployed: no




![](Decision Tree Plots/ID3.png)

## The C4.5 Tree

In [31]:
c45_tree = C45.c45(sample, 'y', features)
print(treeify(c45_tree))


housing: 
 | no: 
 | | poutcome: 
 | | | failure: yes
 | | | success: yes
 | | | unknown: 
 | | | | month: 
 | | | | | apr: yes
 | | | | | aug: yes
 | | | | | jul: no
 | | | | | jun: 
 | | | | | | job: 
 | | | | | | | admin.: 
 | | | | | | | | marital: 
 | | | | | | | | | married: yes
 | | | | | | | | | single: no
 | | | | | | | | 
 | | | | | | | 
 | | | | | | | management: no
 | | | | | | | self-employed: no
 | | | | | | 
 | | | | | 
 | | | | | may: no
 | | | | | nov: no
 | | | | 
 | | | 
 | | 
 | 
 | yes: 
 | | contact: 
 | | | cellular: 
 | | | | marital: 
 | | | | | divorced: yes
 | | | | | married: 
 | | | | | | job: 
 | | | | | | | admin.: yes
 | | | | | | | blue-collar: no
 | | | | | | | housemaid: no
 | | | | | | | management: no
 | | | | | | | self-employed: no
 | | | | | | | services: no
 | | | | | | | technician: yes
 | | | | | | 
 | | | | | 
 | | | | | single: no
 | | | | 
 | | | 
 | | | telephone: no
 | | | unknown: no
 | | 
 | 




![](Decision Tree PLots/C4.5.png)

## The CART Tree

In [45]:
CART_tree = CART(sample, 'y', features)
print(treeify(CART_tree))


job: 
 | admin.: no
 | blue-collar: no
 | entrepreneur: no
 | housemaid: no
 | management: 
 | | education: 
 | | | primary: yes
 | | | secondary: no
 | | | tertiary: no
 | | 
 | 
 | self-employed: no
 | services: 
 | | month: 
 | | | jul: no
 | | | jun: no
 | | | may: 
 | | | | marital: 
 | | | | | married: yes
 | | | | | single: no
 | | | | 
 | | | 
 | | | nov: no
 | | 
 | 
 | student: no
 | technician: no
 | unemployed: no




![](Decision Tree PLots/CART.png)

In [47]:
# graph = pydot.Dot(graph_type='graph')
# visit(tree)
# graph.write_png('CART.png')